In [1]:
from dataclasses import dataclass, asdict, fields
from typing import Optional
import numpy as np
import pandas as pd
import plotly.express as px
from dataPotter import DataPlotter


## Dataclasses Declaration

In [2]:
@dataclass
class ExecutionEntity:
    Execution_DateTime: np.datetime64
    Endpoint: str
    Run_Time: float

@dataclass
class Artifact:
    Name: str
    Execution_DateTime: np.datetime64

@dataclass
class ArtifactInfo:
    Artifact_Name: str
    URL: Optional[str]   #url to the object stored on the cloud, can be null
    Artifact_Type: str
    File_Extension: str

@dataclass
class Tests:
    Artifact_Name: str
    Name: str
    Category: str
    Status: str
    Arguments: Optional[str]  # Sometimes details the arguments they receive 
    Execution_DateTime: np.datetime64

@dataclass
class ExecutionTime:
    Execution_name: str  # Can be Test_Name, fixture, or function
    Execution_type: str
    Execution_DateTime: np.datetime64
    Number_Runs: int
    Avg_Time: float
    Min_Time: float
    Total_Time: float

@dataclass
class Failures:
    Artifact_Name: str
    Test_Name: str
    Execution_DateTime: np.datetime64
    Error: str
    Details: Optional[str]  # Retrieved pytest error description

@dataclass
class TestData:
    execution_entity: list[ExecutionEntity]
    artifact: list[Artifact]
    artifact_info: list[ArtifactInfo]
    tests: list[Tests]
    execution_time: list[ExecutionTime]
    failures: list[Failures]

def get_field_names(Dataclass: type) -> list[str]:
    return [field.name for field in fields(Dataclass)]


In [11]:
# Mock data for tests
tests_data = {
    'Test_Name': ['Test1', 'Test1', 'Test2', 'Test2', 'Test3', 'Test3'],
    'Name': ['Test1', 'Test1', 'Test2', 'Test2', 'Test3', 'Test3'],
    'Category': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Status': ['PASSED', 'FAILED', 'PASSED', 'FAILED', 'PASSED', 'FAILED'],
    'Execution_DateTime': [pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-03'), pd.to_datetime('2023-01-03')],
    'Arguments': [None, None, None, None, None, None]
}

tests_df = pd.DataFrame(tests_data)

# Mock data for failures
failures_data = {
    'Test_Name': ['Test1', 'Test2', 'Test3'],
    'Error': ['Error1', 'Error2', 'Error3'],
    'Execution_DateTime': [pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-03')],
    'Details': [None, None, None]
}

failures_df = pd.DataFrame(failures_data)

# Mock data for execution time
execution_time_data = {
    'Execution_name': ['Test1', 'Test2', 'Test3'],
    'Execution_type': ['Test', 'Test', 'Test'],
    'Execution_DateTime': [pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-03')],
    'Number_Runs': [1, 2, 3],
    'Avg_Time': [10.0, 20.0, 30.0],
    'Min_Time': [5.0, 10.0, 15.0],
    'Total_Time': [10.0, 40.0, 90.0]
}

execution_time_df = pd.DataFrame(execution_time_data)

# Mock data for artifact info
artifact_info_data = {
    'Artifact_Name': ['Artifact1', 'Artifact2', 'Artifact3'],
    'URL': ['https://example.com/artifact1', None, 'https://example.com/artifact3'],
    'Artifact_Type': ['Type1', 'Type2', 'Type3'],
    'File_Extension': ['.txt', '.pdf', '.docx']
}

artifact_info_df = pd.DataFrame(artifact_info_data)

# Mock data for execution entity
execution_entity_data = {
    'Execution_DateTime': [pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-03')],
    'Endpoint': ['Endpoint1', 'Endpoint2', 'Endpoint3'],
    'Run_Time': [10.0, 20.0, 30.0]
}

execution_entity_df = pd.DataFrame(execution_entity_data)

# Mock data for artifacts
artifacts_data = {
    'Name': ['Artifact1', 'Artifact2', 'Artifact3'],
    'Execution_DateTime': [pd.to_datetime('2023-01-01'), pd.to_datetime('2023-01-02'), pd.to_datetime('2023-01-03')]
}

artifacts_df = pd.DataFrame(artifacts_data)

# Create a TestData object
class TestData:
    def __init__(self, execution_entity, artifact, artifact_info, tests, execution_time, failures):
        self.execution_entity = execution_entity
        self.artifact = artifact
        self.artifact_info = artifact_info
        self.tests = tests
        self.execution_time = execution_time
        self.failures = failures
        print(failures)

test_data = TestData(execution_entity_df, artifacts_df, artifact_info_df, tests_df, execution_time_df, failures_df)
# Now you can use test_data to test your function




  Test_Name   Error Execution_DateTime Details
0     Test1  Error1         2023-01-01    None
1     Test2  Error2         2023-01-02    None
2     Test3  Error3         2023-01-03    None


In [10]:
d = DataPlotter(test_data)
#d.test_name_error_distribution_pie_chart()
d.plot_category_errors_bar()
#d.categories_failures_passed_rate()

## Schemes

Execution Pipeline: 
    1. Script gera os dados de datetime e total time


    Script > Execution_DateTime, Run_Time
    Run Tests or download artifacts (Github)
    Put generated artifacts into a bucket adressing essential info into its name
    Extract the test, execution time and failures info out of the pytest_artifact


Note:   It might be helpful to create a Run_Id
        the summary does not to have a run id since it will be used mostly for individual runs

__create_df__,__get_time__: classes: ExecutionTime, Tests (Name, Category) #ExecutionTime Name is a FK from Test (Name)

__create_title__: ExecutionEntity

__create_errors_summary__: classes: Failures, Test(Name, Category)

__PdfDataPlotter__: This one is the generator of graphics it will need more than whats expected from the PdfMaker class, so it should be called earlier, generate the graphics as useful images which will be retrieved and added on the pdf

## Dataclasses DFs Declaration

In [5]:
# Create an empty instance (or use default values)
empty_execution_entity = ExecutionEntity(
    Execution_DateTime=np.datetime64("NaT"),  # Not a Time
    Endpoint="",
    Run_Time=0.0
)

# Convert to DataFrame
df_execution_entity = pd.DataFrame([asdict(empty_execution_entity)])
df_execution_entity.dtypes

Execution_DateTime    datetime64[ns]
Endpoint                      object
Run_Time                     float64
dtype: object